This script creates watersheds for stream temperature sites being used for the AYK project. These include any locations within watersheds for the different populations or genetic substocks and also some migration routes.

Using the MERIT-Basins stream network and catchments. These were created
using a 90m DEM and TauDEM.

NOTE: May need to review point locations, first try see if they fall correctly within a merit-basins catchment

1. intersect points with catchments to get catchment ID
2. create list of IDs for which we want watersheds
3. use loop to identify upstream catchments
4. merge catchments to create watersheds
5. use spatial join to get location names onto watersheds (location_short matches site spreadsheet created by Megan on google drive -- her populations are linked to major spawning areas).




In [5]:
# get list of catchment ids for sites

import arcpy
import pandas as pd

ayk_chinook_gdb = r"W:\GIS\AYK_Chinook\AYK_Chinook.gdb"
arcpy.env.workspace = ayk_chinook_gdb
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(4326)  #'WGS 1984'
arcpy.env.outputCoordinateSystem = sr

points = ayk_chinook_gdb + "\\temp_33sites"
cats = r"W:\GIS\MERIT-Basins\cat_pfaf_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp"
idList = []
outcats = ayk_chinook_gdb + "\\cats_tempSites_int"

arcpy.MakeFeatureLayer_management(points, "pointsTempLayer")
arcpy.MakeFeatureLayer_management(cats, "tempLayer")
arcpy.SelectLayerByLocation_management("templayer", "INTERSECT", "pointsTempLayer")
arcpy.CopyFeatures_management("templayer", outcats)

fields = arcpy.ListFields("tempLayer")
for field in fields:
    print("{0}".format(field.name))
with arcpy.da.SearchCursor("tempLayer", ["COMID"]) as cursor:
    for row in cursor:
        idList.append(row[0])

print(len(idList))


FID
Shape
COMID
unitarea
33


In [6]:
# Find upstream COMIDs and merge to create watersheds in memory and put in a list


import os
import arcpy
import pandas as pd

ayk_chinook_gdb = r"W:\GIS\AYK_Chinook\AYK_Chinook.gdb"
arcpy.env.workspace = ayk_chinook_gdb
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(4326)  #'WGS 1984'
arcpy.env.outputCoordinateSystem = sr

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()
streams = r"W:\GIS\MERIT-Basins\riv_pfaf_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp"
cats = r"W:\GIS\MERIT-Basins\cat_pfaf_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp"

# Get list of index names for cats and add index if not already created
index_names = [i.name for i in arcpy.ListIndexes(cats)]
print(index_names)
if 'comid_index' not in index_names:
    print (f'Creating index for {cats}')
    arcpy.AddIndex_management(cats, "COMID", "comid_index")
else:
    print(f'{cats} Indexed')

fields = arcpy.ListFields(streams)
for field in fields:
    print("{0}".format(field.name))

str_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(streams, ("COMID", "up1", "up2", "up3")))
hws_codes = [0]

#idList if doing ALL watersheds.
wtds_out = list()
c = 1
for id in idList:
    print(f"{c}. Starting watershed for: " + str(id))
    rec = [id]
    up_ids = []
    sum_rec = sum(rec)

    while(sum_rec > 0):
        up_ids.append(rec)
        rec = str_df.loc[str_df["COMID"].isin(rec), ("up1", "up2", "up3")]
        rec = pd.concat([rec['up1'], rec['up2'], rec['up3']])
        sum_rec = sum(rec)

    #up_ids is a list with more than numbers, use extend to only keep numeric comids
    newup_ids = []
    for x in up_ids:
        newup_ids.extend(x)

    tempLayer = "catsLyr"
    expression = '"COMID" IN ({0})'.format(', '.join(map(str, newup_ids)) or 'NULL')
    arcpy.MakeFeatureLayer_management(cats, tempLayer)
    arcpy.management.SelectLayerByAttribute(tempLayer, "NEW_SELECTION", expression, None)
    print("Starting dissolve")
    #outdis = "memory/wtd_" + str(round(id))
    outwtd = os.path.join('in_memory', "wtd_" + str(round(id)))
    arcpy.Dissolve_management(tempLayer, outwtd)
    #watershed = arcpy.EliminatePolygonPart_management(dis, outwtd,"PERCENT", "0 SquareKilometers", 90, "CONTAINED_ONLY")
    arcpy.AddField_management(outwtd,"outlet_comid")
    arcpy.CalculateField_management(outwtd, "outlet_comid", id)
    print("Watershed created at:" + outwtd)
    wtds_out.append(outwtd)
    c=c+1

print(wtds_out)

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')


['COMID']
Creating index for W:\GIS\MERIT-Basins\cat_pfaf_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp
FID
Shape
COMID
lengthkm
lengthdir
sinuosity
slope
uparea
order
strmDrop_t
slope_taud
NextDownID
maxup
up1
up2
up3
up4
1. Starting watershed for: 81034404.0
Starting dissolve
Watershed created at:in_memory\wtd_81034404
2. Starting watershed for: 81007648.0
Starting dissolve
Watershed created at:in_memory\wtd_81007648
3. Starting watershed for: 81007086.0
Starting dissolve
Watershed created at:in_memory\wtd_81007086
4. Starting watershed for: 81010487.0
Starting dissolve
Watershed created at:in_memory\wtd_81010487
5. Starting watershed for: 81010247.0
Starting dissolve
Watershed created at:in_memory\wtd_81010247
6. Starting watershed for: 81015189.0
Starting dissolve
Watershed created at:in_memory\wtd_81015189
7. Starting watershed for: 81019697.0
Starting dissolve
Watershed created at:in_memory\wtd_81019697
8. Starting watershed for: 81025018.0
Starting dissolve
Watershed created at:in_me

In [7]:
# Merge all watersheds into one feature class using list from code chunk above.
import arcpy
arcpy.env.overwriteOutput = True

arcpy.env.workspace = ayk_chinook_gdb
wtd_output = "wtds_tempSites_merge"
wtd_merge = arcpy.Merge_management(wtds_out, wtd_output)

print('Process complete')

Process complete


In [9]:
# Get fields from points feature class onto watersheds
# 1. spatial join between points and comid in catchments.
# 2. use join fields to merge point attributes to watersheds using comids.

import arcpy
import pandas as pd

ayk_chinook_gdb = r"W:\GIS\AYK_Chinook\AYK_Chinook.gdb"
arcpy.env.workspace = ayk_chinook_gdb
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(4326)  #'WGS 1984'
arcpy.env.outputCoordinateSystem = sr

points = os.path.join(ayk_chinook_gdb, "temp_33sites")
out_pts = os.path.join(ayk_chinook_gdb, "temp_33sites_comid")
wtds = os.path.join(ayk_chinook_gdb, "wtds_tempSites_merge")
cats = os.path.join(ayk_chinook_gdb, "cats_tempSites_int")


arcpy.MakeFeatureLayer_management(points, "pointsTempLayer")

arcpy.SpatialJoin_analysis("pointsTempLayer", cats, out_pts)
arcpy.management.JoinField(wtds, "outlet_comid", out_pts, "COMID", "SiteID;Waterbody_name", "NOT_USE_FM", None)

arcpy.management.JoinField(cats, "COMID", out_pts, "COMID", "SiteID;Waterbody_name", "NOT_USE_FM", None)



<Result 'W:\\GIS\\AYK_Chinook\\AYK_Chinook.gdb\\cats_tempSites_int'>